In [12]:
import yaml
class ConfigObject:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            if isinstance(value, dict):
                value = ConfigObject(value)
            self.__dict__[key] = value

    def __repr__(self):
        return f"{self.__dict__}"

def convert_path_to_filepath(base_path, module, value):
    # Convert path like 'mvsplat/encoder: costvolume' to 'mvsplat/encoder/costvolume.yaml'
    return f"{base_path}/{module[0]}/{module[1]}/{value}.yaml"

def load_yaml_config(config_path):
    with open(config_path, 'r') as file:
        config_dict = yaml.safe_load(file)
    return config_dict

def load_config(config_path):
    config_dict = load_yaml_config(config_path)
    for entry in config_dict['defaults']:
        for key, value in entry.items():
            # Split the key on '/' and load corresponding YAML file
            parts = key.split('/')
            module, attribute = parts[0], parts[1]
            yaml_file_path = convert_path_to_filepath('./configs', parts, value)
            additional_config = load_yaml_config(yaml_file_path)
            
            # Place the loaded configuration into a structured object
            if module not in config_dict:
                config_dict[module] = {}
            config_dict[module][attribute] = additional_config
    return ConfigObject(config_dict)


# Load the configuration
config = load_config('../GGRtPlus/configs/finetune_mvsplat_stable.yaml')



In [2]:
import torch
model1 = torch.load("out/generalized_mvspla_view_4/model/model_008000.pth")
model2 = torch.load("model_zoo/mvsplat.pth")

In [5]:
model2['model'] = model1['gaussian']

In [6]:
torch.save(model2, 'model_zoo/mvsplat.pth')